In [ ]:
import os
import pathlib

import numpy as np
import matplotlib.pyplot as plt

from anguilla.fitness import benchmark
from anguilla.evaluation import load_logs

import anguilla.hypervolume as hv
from anguilla.dominance import NonDominatedSet2D, NonDominatedSetKD

In [ ]:
FNS_2D = [
    'ZDT1',
    'ZDT2',
    'ZDT3',
    'ZDT4',
    'ZDT6',
    'IHR1',
    'IHR2',
    'IHR3',
    'IHR4',
    'IHR6',
    'ELLI1',
    'ELLI2',
    'CIGTAB1',
    'CIGTAB2'
]

FNS_3D = [
    'DTLZ1',
    'DTLZ2',
    'DTLZ3',
    'DTLZ4',
    'DTLZ5',
    'DTLZ6',
    'DTLZ7',
    'GELLI3',
]

In [ ]:
def plot_step_size_history(logs):
    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(111)
    data = {}
    key = None
    for log in sorted(logs, key=lambda log: (log.optimizer, log.n_evaluations, log.trial)):
        if key is None:
            key = (log.optimizer, log.n_evaluations)
        if key != (log.optimizer, log.n_evaluations):
            data[key[0]][key[1]] = np.median(data[key[0]][key[1]])
            key = (log.optimizer, log.n_evaluations)
        if log.optimizer not in data:
            data[log.optimizer] = {}
        if log.n_evaluations not in data[log.optimizer]:
            data[log.optimizer][log.n_evaluations] = []
        data[log.optimizer][log.n_evaluations] += log.data.tolist()
    data[key[0]][key[1]] = np.median(data[key[0]][key[1]])
    ticks_set = False
    for opt, history in data.items():
        ax.plot(history.keys(), history.values(), marker='s', label=opt)
        if not ticks_set:
            ax.set_xticks(list(history.keys()))
            ticks_set = True
    ax.set_title(logs[0].fn)
    ax.set_ylabel('Step size (median)')
    ax.set_xlabel('Function evaluations')
    ax.legend()
    fig.tight_layout()
    return fig

In [ ]:
for fn in FNS_2D + FNS_3D:
    print(f"Processing {fn}...")
    path = pathlib.Path("./plots/step_size")
    if not path.exists():
        os.makedirs(path, exist_ok=True)
    logs = load_logs("./data/anguilla", fns=[fn], observations=["step_sizes"], search_subdirs=True)
    fig = plot_step_size_history(logs)
    fig.savefig(path.joinpath(f"{fn}.pdf"), bbox_inches="tight")
    fig.savefig(path.joinpath(f"{fn}.png"), bbox_inches="tight")
    plt.close(fig)